In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

In [2]:
def obtener_rmse(col_true, col_pred):
    return mean_squared_error(col_true, col_pred)**0.5

In [3]:
propiedades = pd.read_csv('/home/agustin/Escritorio/escritorio/fiuba/Organizacion de datos/datos para el tp2/set_datos_propiedades.csv')

In [12]:
propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull()),\
                             ['place_name_encoded', 'property_type_encoded','price_aprox_usd','superficie',\
                             'Year','Month','seguridad','aire','gimnasio','cochera','pileta']]

In [13]:
propiedades.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1132495 entries, 0 to 1413024
Data columns (total 11 columns):
place_name_encoded       1132495 non-null int64
property_type_encoded    1132495 non-null int64
price_aprox_usd          1132495 non-null float64
superficie               1132495 non-null float64
Year                     1132495 non-null int64
Month                    1132495 non-null int64
seguridad                1132495 non-null bool
aire                     1132495 non-null bool
gimnasio                 1132495 non-null bool
cochera                  1132495 non-null bool
pileta                   1132495 non-null bool
dtypes: bool(5), float64(2), int64(4)
memory usage: 65.9 MB


# Gradient boosting regressor

In [14]:
columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [15]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [21]:
gradient = GradientBoostingRegressor()
gradient.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: gradient.predict([x])[0],axis = 1)
precision = gradient.score(set_pruebas.loc[:,columnas], set_pruebas.price_aprox_usd) * 100
error = obtener_rmse(set_pruebas.price_aprox_usd, set_pruebas.resultado)
print("Precision = {:.2f} % , error = {}".format(precision, error))

/usr/local/lib/python2.7/dist-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


Precision = -260.00 % , error = 704737.303778


#### Al correrlo varias veces con distintos valores, fue mejorando la precision. Despues agrego un for con varios valores para ver como mejora

## Ahora que tenemos una intuicion, probamos cambiando los parametros

In [22]:
columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [23]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

res = []

In [25]:
loss = ['ls', 'lad', 'huber', 'quantile']
n_estimators = [100, 200, 300]

for l in loss:
    for n in n_estimators:
        gradient = GradientBoostingRegressor(loss = l, n_estimators = n)
        gradient.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
        set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: gradient.predict([x])[0],axis = 1)
        precision = gradient.score(set_pruebas.loc[:,columnas],set_pruebas.loc[:,'price_aprox_usd']) * 100
        error = obtener_rmse(set_pruebas.price_aprox_usd, set_pruebas.resultado)
        res.append((l, n, precision, error))
        print(k,' - ', t)

('linear', ' - ', 0.001)
('linear', ' - ', 1e-05)
('linear', ' - ', 1e-07)
('poly', ' - ', 0.001)
('poly', ' - ', 1e-05)
('poly', ' - ', 1e-07)
('rbf', ' - ', 0.001)
('rbf', ' - ', 1e-05)
('rbf', ' - ', 1e-07)
('sigmoid', ' - ', 0.001)
('sigmoid', ' - ', 1e-05)
('sigmoid', ' - ', 1e-07)


In [26]:
for r in res:
    print("Loss = {}, n_estimators = {}, precision = {:.2f} % , error = {}".format(r[0],r[1],r[2],r[3]))

Kernel = linear, tolerancia = 0.001, precision = -19157626221918900.00 % , error = 5.14100173151e+12
Kernel = linear, tolerancia = 1e-05, precision = -19157626221918900.00 % , error = 5.14100173151e+12
Kernel = linear, tolerancia = 1e-07, precision = -19157626221918900.00 % , error = 5.14100173151e+12
Kernel = poly, tolerancia = 0.001, precision = -52373501570356460102347948475995973293908888777264505032147992576.00 % , error = 8.50026974531e+36
Kernel = poly, tolerancia = 1e-05, precision = -52373501570356460102347948475995973293908888777264505032147992576.00 % , error = 8.50026974531e+36
Kernel = poly, tolerancia = 1e-07, precision = -52373501570356460102347948475995973293908888777264505032147992576.00 % , error = 8.50026974531e+36
Kernel = rbf, tolerancia = 0.001, precision = -260.00 % , error = 704737.303778
Kernel = rbf, tolerancia = 1e-05, precision = -260.00 % , error = 704737.303778
Kernel = rbf, tolerancia = 1e-07, precision = -260.00 % , error = 704737.303778
Kernel = sigmoid

In [31]:
min_error = float('inf')
max_precision = 0
tupla_min_error = ()
tupla_max_precision = ()
for r in res:
    if r[3] < min_error:
        min_error = r[3]
        tupla_min_error = r
    if abs(r[2]) > max_precision:
        max_precision = r[2]
        tupla_max_precision = r
        
print("Mayor precision = Loss = {}, n_estimators = {}, precision = {:.2f} % , error = {}".\
              format(tupla_max_precision[0],tupla_max_precision[1],tupla_max_precision[2],tupla_max_precision[3]))
print("Menor error = Loss = {}, n_estimators = {}, precision = {:.2f} % , error = {}".\
              format(tupla_min_error[0],tupla_min_error[1],tupla_min_error[2],tupla_min_error[3]))

Mayor precision = Kernel = sigmoid, tolerancia = 1e-07, precision = -259.16 % , error = 703916.898926
Menor error = Kernel = sigmoid, tolerancia = 0.001, precision = -259.16 % , error = 703916.898926


('sigmoid', 0.001, -259.16083967299721, 703916.89892584097)

## Usamos sigmoid y rbf como kernel y variamos el maximos de iteraciones

In [34]:
columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [35]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [36]:
kernel = ['rbf', 'sigmoid']
iteraciones = [5000, 10000]

In [39]:
for k in kernel:
    for i in iteraciones:
        svr = SVR(kernel = k, max_iter = i, tol = 1e-3)
        svr.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
        set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: svr.predict([x])[0],axis = 1)
        precision = svr.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
        error = obtener_rmse(set_pruebas.price_aprox_usd,set_pruebas.resultado)
        print("Precision = {:.2f} % , error = {}".format(precision, error))

Precision = -260.00 % , error = 704737.303778


/usr/local/lib/python2.7/dist-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


Precision = -131.22 % , error = 564795.022695
Precision = -259.16 % , error = 703916.898926
Precision = -120.48 % , error = 551519.644372


#### sigmoid parece ser mejor. Voy a utilizar standard scaler como recomienda el error. 

In [42]:
from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()  
scaler.fit(set_entrenamiento_datos)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [43]:
for i in iteraciones:
    svr = SVR(kernel = 'sigmoid', max_iter = i, tol = 1e-3)
    svr.fit(scaler.transform(set_entrenamiento_datos),set_entrenamiento_resultado)
    set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas]\
    .apply(lambda x: svr.predict(scaler.transform([x]))[0],axis = 1)
    precision = svr.score(scaler.transform(set_pruebas.loc[:,columnas]),set_pruebas.price_aprox_usd) * 100
    error = obtener_rmse(set_pruebas.price_aprox_usd,set_pruebas.resultado)
    print("Precision = {:.2f} % , error = {}".format(precision, error))

Precision = -249.48 % , error = 694362.851523
Precision = -108.38 % , error = 536173.740674


In [ ]:
svr = SVR(kernel = 'sigmoid', max_iter = 100000, tol = 1e-3)
svr.fit(scaler.transform(set_entrenamiento_datos),set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas]\
.apply(lambda x: svr.predict(scaler.transform([x]))[0],axis = 1)
precision = svr.score(scaler.transform(set_pruebas.loc[:,columnas]),set_pruebas.price_aprox_usd) * 100
error = obtener_rmse(set_pruebas.price_aprox_usd,set_pruebas.resultado)
print("Precision = {:.2f} % , error = {}".format(precision, error))

# calculo con los verdaderos datos a analizar

In [25]:
analizar = pd.read_csv("/home/mati/Desktop/properati_dataset_modificado.csv")

In [45]:
analizar.loc[:,'price_usd'] = analizar.loc[:,columnas].apply(lambda x: randomforest.predict(x)[0],axis = 1)

In [26]:
resultado = analizar.loc[:,['id','price_usd']]

In [ ]:
resultado.to_csv('Random_Forests_resultados', index = False)